In [74]:
import numpy as np
from itertools import combinations
import sympy as sp






In [6]:
def generate_basis_states():
    """
    Generate all possible states for 2 electrons in 4 orbitals.
    Returns: List of binary strings like ['1100', '1010', ...]
    """
    basis_states = []
    
    # We have 4 orbitals (0,1,2,3): 2 sites × 2 spins
    # Choose 2 orbitals to be occupied (2 electrons)
    for occupied in combinations(range(4), 2):
        # Start with all zeros: ['0', '0', '0', '0']
        state = ['0'] * 4
        # Set the occupied orbitals to '1'
        for pos in occupied:
            state[pos] = '1'
        
        # Convert list to string: ['1','1','0','0'] → "1100"
        state_str = ''.join(state)
        basis_states.append(state_str)
    
    return basis_states

In [7]:
# Return the physical information for the input basis state
def basis_state_info(basis_state):
    up1 = int(basis_state[0])    # site 1, spin up
    down1 = int(basis_state[1])  # site 1, spin down
    up2 = int(basis_state[2])    # site 2, spin up
    down2 = int(basis_state[3])  # site 2, spin down

    if up1 == 1 and down1 == 1:
        site1_desc = "↑↓ (doubly occupied)"
    elif up1 == 1:
        site1_desc = "↑"
    elif down1 == 1:
        site1_desc = "↓"
    else:
        site1_desc = "empty"

    if up2 == 1 and down2 == 1:
        site2_desc = "↑↓ (doubly occupied)"
    elif up2 == 1:
        site2_desc = "↑"
    elif down2 == 1:
        site2_desc = "↓"
    else:
        site2_desc = "empty"
    
    print(f"{basis_state}")
    print(f"  Site 1: {site1_desc}")
    print(f"  Site 2: {site2_desc}")


In [8]:
# Map the basis states to state indices to construct the Hamiltonian matrix
def state_index_mapping(basis_states):
    """
    Create dictionary mapping state strings to matrix indices.
    Example: {"1100": 0, "1010": 1, ...}
    """
    state_idx = {}
    for i, state in enumerate(basis_states):
        state_idx[state] = i
    return state_idx

In [125]:
def Hubbard_Hamiltonian_matrix(basis_states,t,U):
    state_dict = state_index_mapping(basis_states)
    matrix = sp.zeros(6)
    
    for i in basis_states:
        if i[2]=='1' and i[0]=='0':
            temp_list=list(i)
            temp_list[2]='0'
            temp_list[0]='1'
            temp_string=''.join(temp_list)
            column=state_dict[i]
            row=state_dict[temp_string]
            matrix[row, column]=-t

        if i[0]=='1' and i[2]=='0':
            temp_list=list(i)
            temp_list[0]='0'
            temp_list[2]='1'
            temp_string=''.join(temp_list)
            column=state_dict[i]
            row=state_dict[temp_string]
            matrix[row, column]=-t

        if i[3]=='1' and i[1]=='0':
            temp_list=list(i)
            temp_list[3]='0'
            temp_list[1]='1'
            temp_string=''.join(temp_list)
            column=state_dict[i]
            row=state_dict[temp_string]
            matrix[row, column]=-t   

        if i[1]=='1' and i[3]=='0':
            temp_list=list(i)
            temp_list[1]='0'
            temp_list[3]='1'
            temp_string=''.join(temp_list)
            column=state_dict[i]
            row=state_dict[temp_string]
            matrix[row, column]=-t   


        if i[0]=='1' and i[1]=='1':
            column=state_dict[i]
            row=state_dict[i]
            matrix[row, column]= U

        if i[2]=='1' and i[3]=='1':
            column=state_dict[i]
            row=state_dict[i]
            matrix[row, column]= U


    return matrix



            


    


In [126]:
basis=generate_basis_states()

#print(state_index_mapping(basis))
t, U = sp.symbols('t U', real=True)
HH_matrix=Hubbard_Hamiltonian_matrix(basis,t,U)
print(HH_matrix)


Matrix([[U, 0, -t, -t, 0, 0], [0, 0, 0, 0, 0, 0], [-t, 0, 0, 0, 0, -t], [-t, 0, 0, 0, 0, -t], [0, 0, 0, 0, 0, 0], [0, 0, -t, -t, 0, U]])


In [127]:


eigenvects = HH_matrix.eigenvects()

print("\nEigenvectors with corresponding eigenvalues:")
for eigval, multiplicity, eigenvectors in eigenvects:
    print(f"\nEigenvalue: λ = {sp.simplify(eigval)}")
    
    for i, vec in enumerate(eigenvectors):
        # vec is a Matrix column vector
        # Simplify and show as expressions of t and U
        vec_simplified = sp.simplify(vec)
        print(f"  Eigenvector {i+1}:")
        print(f"    {vec_simplified.T}")  

"""

P, D = H_matrix.diagonalize()

print("\nMatrix P (eigenvectors as columns):")
sp.pprint(P)

print("\nEach eigenvector column explicitly:")
for i in range(P.shape[1]):
    eigenvector_i = P[:, i]  # Get i-th column
    eigenvector_i_simplified = sp.simplify(eigenvector_i)
    print(f"\nv_{i+1} (for λ = {sp.simplify(D[i,i])}):")
    print(f"  {eigenvector_i_simplified.T}")


print(type(H_matrix))



"""






Eigenvectors with corresponding eigenvalues:

Eigenvalue: λ = 0
  Eigenvector 1:
    Matrix([[0, 1, 0, 0, 0, 0]])
  Eigenvector 2:
    Matrix([[0, 0, -1, 1, 0, 0]])
  Eigenvector 3:
    Matrix([[0, 0, 0, 0, 1, 0]])

Eigenvalue: λ = U
  Eigenvector 1:
    Matrix([[-1, 0, 0, 0, 0, 1]])

Eigenvalue: λ = U/2 - sqrt(U**2 + 16*t**2)/2
  Eigenvector 1:
    Matrix([[1, 0, (U + sqrt(U**2 + 16*t**2))/(4*t), (U + sqrt(U**2 + 16*t**2))/(4*t), 0, 1]])

Eigenvalue: λ = U/2 + sqrt(U**2 + 16*t**2)/2
  Eigenvector 1:
    Matrix([[1, 0, (U - sqrt(U**2 + 16*t**2))/(4*t), (U - sqrt(U**2 + 16*t**2))/(4*t), 0, 1]])


'\n\nP, D = H_matrix.diagonalize()\n\nprint("\nMatrix P (eigenvectors as columns):")\nsp.pprint(P)\n\nprint("\nEach eigenvector column explicitly:")\nfor i in range(P.shape[1]):\n    eigenvector_i = P[:, i]  # Get i-th column\n    eigenvector_i_simplified = sp.simplify(eigenvector_i)\n    print(f"\nv_{i+1} (for λ = {sp.simplify(D[i,i])}):")\n    print(f"  {eigenvector_i_simplified.T}")\n\n\nprint(type(H_matrix))\n\n\n\n'

In [120]:
print(HH_matrix)


Matrix([[0, 0, -t, -t, 0, 0], [0, 0, 0, 0, 0, 0], [-t, 0, 0, 0, 0, -t], [-t, 0, 0, 0, 0, -t], [0, 0, 0, 0, 0, 0], [0, 0, -t, -t, 0, 0]])
